In [1]:
import numpy as np
import astropy
import ccdproc
from ccdproc import CCDData, Combiner
from astropy import units as u
from astropy.visualization import SqrtStretch
import matplotlib.pyplot as plt 
from matplotlib.colors import LogNorm
from photutils import centroid_com, centroid_1dg, centroid_2dg
from photutils import CircularAperture
from photutils import aperture_photometry
from photutils import Background2D
from photutils import MedianBackground
from scipy.ndimage import shift
import gc                                                           
gc.enable()

C:\Users\Msi Raider\AppData\Local\Temp\ipykernel_14412\1974830583.py:9: DeprecationWarning: `photutils.centroid_com` is a deprecated alias for `photutils.centroids.centroid_com`. Instead, please use `from photutils.centroids import centroid_com` to silence this warning.
  from photutils import centroid_com, centroid_1dg, centroid_2dg
C:\Users\Msi Raider\AppData\Local\Temp\ipykernel_14412\1974830583.py:9: DeprecationWarning: `photutils.centroid_1dg` is a deprecated alias for `photutils.centroids.centroid_1dg`. Instead, please use `from photutils.centroids import centroid_1dg` to silence this warning.
  from photutils import centroid_com, centroid_1dg, centroid_2dg
C:\Users\Msi Raider\AppData\Local\Temp\ipykernel_14412\1974830583.py:9: DeprecationWarning: `photutils.centroid_2dg` is a deprecated alias for `photutils.centroids.centroid_2dg`. Instead, please use `from photutils.centroids import centroid_2dg` to silence this warning.
  from photutils import centroid_com, centroid_1dg, centr

In [2]:
# The same code needed to be excecuted to create flats for the 5 filters that we had observations in.
# TO do this more efficiently we decided to create a function that could be called 
# When called it returns the combined flat image that has had the bias and dark subtracted from it
# Along with the statistics before and after the combining to show that the image has become a lot more uniform

def median_flat_creator(band):
    string_for_loading = 'Flat_' + band + '_*'
    string_for_saving = "Flat_" + band + "_median.fits"
    
    images = ccdproc.ImageFileCollection(".",glob_include = string_for_loading)#loading in image file collection
    for fn in images.files_filtered(PICTTYPE = 4): #Printing file names
        #print(fn)
        pass
    flats = [ CCDData.read(fn, unit = "adu") for fn in images.files_filtered(PICTTYPE = 4) ] #Loading in Flats data

    print('Min of flats_' + band + "_[0]:", np.min(flats[0]))
    print('Max of flats_' + band + "_[0]:", np.max(flats[0]))
    print('Median of flats_' + band + "_[0]:", np.median(flats[0]))
    print('Std Dev of flats_' + band + "_[0]:", np.std(flats[0]))

    for idx, thisimage in enumerate(flats): 
        flats[idx] = ccdproc.subtract_bias(thisimage, bias_median) #Subtracting Bias from flats

    for idx, thisimage in enumerate(flats): 
        flats[idx] = ccdproc.subtract_dark(thisimage, dark_median, exposure_time = 'EXPTIME', #Subtracting darks from flats
                                           exposure_unit = u.second, scale = True)

    tempimages = flats.copy() #copying the flat images into a temporary variable
    for idx, thisimage in enumerate(tempimages):
        m = 1.0 / np.ma.median(tempimages[idx]) #finding the scaling for each image based of it's median counts
        tempimages[idx] = tempimages[idx].multiply(m * u.adu) #scaling each image by the scaling factor that was found before
    Flat_median = ccdproc.Combiner(tempimages, dtype=np.float32).median_combine() #combining the scaled images into 1 median image
    print("")
    print('Min of flats_' + band + '_median combine image:' , np.min(Flat_median))
    print('Max of flats_' + band + '_median combine image:', np.max(Flat_median))
    print('Median of flats_' + band + '_median combine image:', np.median(Flat_median))
    print('Std Dev of flats_' + band + '_median combine image:', np.std(Flat_median))

    Flat_median.write(string_for_saving)

In [4]:
# Reading in the previously created bias and dark frames

bias_median = CCDData.read("bias_median.fits", unit = "adu")
dark_median = CCDData.read("dark_median.fits", unit = "adu")

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


In [5]:
# Calling the function for the 5 differenct bands
bands = ["B", "V", "R", "I", "U"]

for i in range(0,5):
    median_flat_creator(bands[i])

Set OBSGEO-Y to  2879792.379 from OBSGEO-[LBH].
Set OBSGEO-Z to -3897419.410 from OBSGEO-[LBH]'. [astropy.wcs.wcs]
Set OBSGEO-Y to  2879792.379 from OBSGEO-[LBH].
Set OBSGEO-Z to -3897419.410 from OBSGEO-[LBH]'.


Min of flats_B_[0]: 13366
Max of flats_B_[0]: 24187
Median of flats_B_[0]: 18734.0
Std Dev of flats_B_[0]: 443.69840937499634

Min of flats_B_median combine image: 0.68633604
Max of flats_B_median combine image: 1.3592789
Median of flats_B_median combine image: 1.0000075101852417
Std Dev of flats_B_median combine image: 0.023409750312566757


C:\Users\Msi Raider\anaconda3\envs\obsastro2022\lib\site-packages\numpy\core\fromnumeric.py:755: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
Set OBSGEO-Y to  2879792.379 from OBSGEO-[LBH].
Set OBSGEO-Z to -3897419.410 from OBSGEO-[LBH]'. [astropy.wcs.wcs]
Set OBSGEO-Y to  2879792.379 from OBSGEO-[LBH].
Set OBSGEO-Z to -3897419.410 from OBSGEO-[LBH]'.


Min of flats_V_[0]: 6014
Max of flats_V_[0]: 12801
Median of flats_V_[0]: 8350.0
Std Dev of flats_V_[0]: 189.37751597457034

Min of flats_V_median combine image: 0.6129931
Max of flats_V_median combine image: 1.7178612
Median of flats_V_median combine image: 1.000342845916748
Std Dev of flats_V_median combine image: 0.02411131002008915


C:\Users\Msi Raider\anaconda3\envs\obsastro2022\lib\site-packages\numpy\core\fromnumeric.py:755: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
Set OBSGEO-Y to  2879792.379 from OBSGEO-[LBH].
Set OBSGEO-Z to -3897419.410 from OBSGEO-[LBH]'. [astropy.wcs.wcs]
Set OBSGEO-Y to  2879792.379 from OBSGEO-[LBH].
Set OBSGEO-Z to -3897419.410 from OBSGEO-[LBH]'.


...

C:\Users\Msi Raider\anaconda3\envs\obsastro2022\lib\site-packages\numpy\core\fromnumeric.py:755: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
Set OBSGEO-Y to  2879792.379 from OBSGEO-[LBH].
Set OBSGEO-Z to -3897419.410 from OBSGEO-[LBH]'. [astropy.wcs.wcs]
Set OBSGEO-Y to  2879792.379 from OBSGEO-[LBH].
Set OBSGEO-Z to -3897419.410 from OBSGEO-[LBH]'.


Min of flats_I_[0]: 4415
Max of flats_I_[0]: 9386
Median of flats_I_[0]: 4958.0
Std Dev of flats_I_[0]: 111.92012230376744

Min of flats_I_median combine image: 0.87500304
Max of flats_I_median combine image: 2.5058527
Median of flats_I_median combine image: 1.00032639503479
Std Dev of flats_I_median combine image: 0.022283578291535378


C:\Users\Msi Raider\anaconda3\envs\obsastro2022\lib\site-packages\numpy\core\fromnumeric.py:755: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
Set OBSGEO-Y to  2879792.379 from OBSGEO-[LBH].
Set OBSGEO-Z to -3897419.410 from OBSGEO-[LBH]'. [astropy.wcs.wcs]
Set OBSGEO-Y to  2879792.379 from OBSGEO-[LBH].
Set OBSGEO-Z to -3897419.410 from OBSGEO-[LBH]'.


Min of flats_U_[0]: 17852
Max of flats_U_[0]: 28207
Median of flats_U_[0]: 22718.0
Std Dev of flats_U_[0]: 635.4923663444304

Min of flats_U_median combine image: 0.76239395
Max of flats_U_median combine image: 1.4133986
Median of flats_U_median combine image: 1.0012414455413818
Std Dev of flats_U_median combine image: 0.029202427715063095


C:\Users\Msi Raider\anaconda3\envs\obsastro2022\lib\site-packages\numpy\core\fromnumeric.py:755: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
